## Validate protein mutations with endstate corrections!!!

this should tell us if we are at all capable of closing cycles...

In [34]:
#!/usr/bin/env python
# coding: utf-8

# # Here, I document an attempt to validate a small set of protein mutations in vacuum and solvent with the following checks...

# 1. generate alanine dipeptide --> valine dipeptide in vac/solvent and conduct a forward _and_ reverse parallel tempering FEP calculation; the check passes if the forward free energy is equal to the reverse free energy within an error tolerance
# 2. generate alanine dipeptide --> valine dipeptide --> isoleucine dipeptide --> glycine dipeptide and attempt to close the thermodynamic cycle within an error tolerance

# In[ ]:


from __future__ import absolute_import

import networkx as nx
from perses.dispersed import feptasks
from perses.utils.openeye import *
from perses.utils.data import load_smi
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from perses.rjmc.topology_proposal import TopologyProposal, TwoMoleculeSetProposalEngine, SystemGenerator,SmallMoleculeSetProposalEngine, PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
import simtk.openmm.app as app
import sys

from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState

import pymbar
import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import pickle
import mdtraj as md
from io import StringIO
from openmmtools.constants import kB
import logging
import os
import dask.distributed as distributed
import parmed as pm
from collections import namedtuple
from typing import List, Tuple, Union, NamedTuple
from collections import namedtuple
import random
#beta = 1.0/(kB*temperature)
import itertools
import os
from openeye import oechem
from perses.utils.smallmolecules import render_atom_mapping
from perses.tests.utils import validate_endstate_energies

ENERGY_THRESHOLD = 1e-6
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.tests.utils import validate_endstate_energies


# In[ ]:


from perses.samplers.multistate import HybridSAMSSampler, HybridRepexSampler
from openmmtools.multistate import MultiStateReporter, MultiStateSamplerAnalyzer
from openmmtools import mcmc, utils
from perses.annihilation.lambda_protocol import LambdaProtocol


# In[ ]:


def generate_atp(phase = 'solvent'):
    """
    modify the AlanineDipeptideVacuum test system to be parametrized with amber14ffsb in vac or solvent (tip3p)
    """
    import openmmtools.testsystems as ts
    atp = ts.AlanineDipeptideVacuum(constraints = app.HBonds, hydrogenMass = 4 * unit.amus)

    forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    
    if phase == 'vacuum':
        barostat = None
        system_generator = SystemGenerator(forcefield_files,
                                       barostat = barostat,
                                       forcefield_kwargs = {'removeCMMotion': False, 
                                                            'ewaldErrorTolerance': 1e-4, 
                                                            'nonbondedMethod': app.NoCutoff,
                                                            'constraints' : app.HBonds, 
                                                            'hydrogenMass' : 4 * unit.amus})
        atp.system = system_generator.build_system(atp.topology) #update the parametrization scheme to amberff14sb
        
    elif phase == 'solvent':
        barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
        system_generator = SystemGenerator(forcefield_files,
                                   barostat = barostat,
                                   forcefield_kwargs = {'removeCMMotion': False, 
                                                        'ewaldErrorTolerance': 1e-4, 
                                                        'nonbondedMethod': app.PME,
                                                        'constraints' : app.HBonds, 
                                                        'hydrogenMass' : 4 * unit.amus})
    
    if phase == 'solvent':
        modeller = app.Modeller(atp.topology, atp.positions)
        modeller.addSolvent(system_generator._forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
        solvated_topology = modeller.getTopology()
        solvated_positions = modeller.getPositions()

        # canonicalize the solvated positions: turn tuples into np.array
        atp.positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
        atp.topology = solvated_topology

        atp.system = system_generator.build_system(atp.topology)
    
    
    return atp, system_generator


# In[ ]:


def generate_top_pos_sys(topology, new_res, system, positions, system_generator):
    """generate point mutation engine, geometry_engine, and conduct topology proposal, geometry propsal, and hybrid factory generation"""
    #create the point mutation engine
    print(f"generating point mutation engine")
    point_mutation_engine = PointMutationEngine(wildtype_topology = topology,
                                                system_generator = system_generator,
                                                chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants = 1,
                                                residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
                                                allowed_mutations = [('2', new_res)], #the residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate = True) #always allow aggregation

    #create a geometry engine
    print(f"generating geometry engine")
    geometry_engine = FFAllAngleGeometryEngine(metadata=None, 
                                           use_sterics=False, 
                                           n_bond_divisions=100, 
                                           n_angle_divisions=180, 
                                           n_torsion_divisions=360, 
                                           verbose=True, 
                                           storage=None, 
                                           bond_softening_constant=1.0, 
                                           angle_softening_constant=1.0, 
                                           neglect_angles = False, 
                                           use_14_nonbondeds = True)

    #create a top proposal
    print(f"making topology proposal")
    topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = system,
                                  current_topology = topology)

    #make a geometry proposal forward
    print(f"making geometry proposal from {list(topology.residues())[1].name} to {new_res}")
    forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta)
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta)


    #create a hybrid topology factory
    f"making forward hybridtopologyfactory"
    forward_htf = HybridTopologyFactory(topology_proposal = topology_proposal,
                 current_positions =  positions,
                 new_positions = forward_new_positions,
                 use_dispersion_correction = False,
                 functions=None,
                 softcore_alpha = None,
                 bond_softening_constant=1.0,
                 angle_softening_constant=1.0,
                 soften_only_new = False,
                 neglected_new_angle_terms = [],
                 neglected_old_angle_terms = [],
                 softcore_LJ_v2 = True,
                 softcore_electrostatics = True,
                 softcore_LJ_v2_alpha = 0.85,
                 softcore_electrostatics_alpha = 0.3,
                 softcore_sigma_Q = 1.0,
                 interpolate_old_and_new_14s = False,
                 omitted_terms = None)
    
    if not topology_proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not topology_proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

#     self._vacuum_forward_neglected_angles = self._geometry_engine.forward_neglected_angle_terms
#     self._vacuum_reverse_neglected_angles = self._geometry_engine.reverse_neglected_angle_terms
#     self._vacuum_geometry_engine = copy.deepcopy(self._geometry_engine)
    
#     zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)
#     print(f"zero state error : {zero_state_error}")
#     print(f"one state error : {one_state_error}")
    
    return topology_proposal, forward_new_positions, forward_htf, local_map_stereo_sidechain, old_oemol_sidechain, new_oemol_sidechain, added_valence_energy, subtracted_valence_energy


# In[ ]:


# def create_hss(reporter_name, hybrid_factory, selection_string ='all', checkpoint_interval = 1, n_states = 13):
#     lambda_protocol = LambdaProtocol(functions='default')
#     reporter = MultiStateReporter(reporter_name, analysis_particle_indices = hybrid_factory.hybrid_topology.select(selection_string), checkpoint_interval = checkpoint_interval)
#     hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
#                                                                                  collision_rate=5.0 / unit.picosecond,
#                                                                                  n_steps=250,
#                                                                                  reassign_velocities=False,
#                                                                                  n_restart_attempts=20,
#                                                                                  splitting="V R R R O R R R V",
#                                                                                  constraint_tolerance=1e-06),
#                                                                                  hybrid_factory=hybrid_factory, online_analysis_interval=10)
#     hss.setup(n_states=n_states, temperature=300*unit.kelvin,storage_file=reporter,lambda_protocol=lambda_protocol,endstates=False)
#     return hss, reporter


# let's make a function to generate an n node graph and run a computation on it...

    


# In[ ]:


def generate_fully_connected_perturbation_graph(dipeptides = ['ALA', 'SER', 'CYS']):
    # generate a fully connected solvation energy graph for the dipeptides specified...
    graph = nx.DiGraph()
    for dipeptide in dipeptides:
        graph.add_node(dipeptide)
    
    #now for edges...
    for i in graph.nodes():
        for j in graph.nodes():
            if i != j:
                graph.add_edge(i, j)
    
    
    #start with ala
    #vac_atp, vac_system_generator = generate_atp(phase = 'vacuum')
    sol_atp, sol_system_generator = generate_atp(phase = 'solvent')
    
    #graph.nodes['ALA']['vac_sys_pos_top'] = (vac_atp.system, vac_atp.positions, vac_atp.topology)
    graph.nodes['ALA']['sol_sys_pos_top'] = (sol_atp.system, sol_atp.positions, sol_atp.topology)
    
    #turn ala into all of the other dipeptides
    for dipeptide in [pep for pep in dipeptides if pep != 'ALA']:
        for phase, testcase, sys_gen in zip(['sol'], [sol_atp], [sol_system_generator]):
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e =  generate_top_pos_sys(testcase.topology, dipeptide, testcase.system, testcase.positions, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[dipeptide][f"{phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[('ALA', dipeptide)][f'{phase}_htf'] = htf
            graph.edges[('ALA', dipeptide)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[('ALA', dipeptide)][f"added_subtracted"] = (added_e, subtracted_e)

        
        
    #now we can turn all of the other states in to each other!!!
    for edge_start, edge_end in list(graph.edges()):
        if edge_start == 'ALA': #we already did ALA
            continue
        
        for phase, sys_gen in zip(['sol'], [sol_system_generator]):
            sys, pos, top = graph.nodes[edge_start][f"{phase}_sys_pos_top"]
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e = generate_top_pos_sys(top, edge_end, sys, pos, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[edge_end][f"{phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[(edge_start, edge_end)][f"{phase}_htf"] = htf
            graph.edges[(edge_start, edge_end)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[(edge_start, edge_end)][f"added_subtracted"] = (added_e, subtracted_e)
            
    print(f"graph_edges: {graph.edges()}")
    
    return graph
        


# In[ ]:


#os.system(f"rm *.nc")

            
        
    

    


In [35]:
mapping_strength = 'default'
import pickle
from perses.utils.smallmolecules import render_atom_mapping
graph = generate_fully_connected_perturbation_graph()
print(f"graph edges: {graph.edges()}")
# for pair in graph.edges():
#     for phase in ['vac', 'sol']:
#         print("Seralizing the system to ", f"{pair}_{phase}" + ".xml")
#         with open(f"{pair[0]}_{pair[1]}.{phase}.{mapping_strength}_map.xml", 'w') as f:
#             hybrid_system = graph.edges[pair][f"{phase}_htf"]._hybrid_system
#             f.write(openmm.openmm.XmlSerializer.serialize(hybrid_system))
        
#         htf = graph.edges[pair][f"{phase}_htf"]
#         htf._topology_proposal._old_networkx_residue.remove_oemols_from_graph()
#         htf._topology_proposal._new_networkx_residue.remove_oemols_from_graph()
#         _map, oldmol, newmol = graph.edges[pair][f"map_oldmol_newmol"]
#         render_atom_mapping(f"{pair[0]}_{pair[1]}.{mapping_strength}_map.png", oldmol, newmol, _map)
#         with open(f"{pair[0]}_{pair[1]}.{phase}.{mapping_strength}_map.pkl", 'wb') as f:
#             pickle.dump(htf, f)

generating point mutation engine
generating geometry engine
making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 5
INFO:geometry:Atom index proposal order is [10, 15, 12, 11, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


making geometry proposal from ALA to SER


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 10 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 37 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 54 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1636 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms inclu

added energy components: [('CustomBondForce', 0.08669105959515), ('CustomAngleForce', 1.8025106863865676), ('CustomTorsionForce', 10.651492770563909), ('CustomBondForce', -31.196092854167027)]


...

making geometry proposal from CYS to SER


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 10 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 37 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 54 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1636 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms inclu

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.001213857654067599), ('CustomTorsionForce', 1.5169217203374377), ('CustomBondForce', 4.613988177710915)]


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 10 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 37 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 54 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1636 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms inclu

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 5.097947770634334), ('CustomTorsionForce', 1.46544785670229), ('CustomBondForce', 9.625616693990791)]
graph_edges: [('ALA', 'SER'), ('ALA', 'CYS'), ('SER', 'ALA'), ('SER', 'CYS'), ('CYS', 'ALA'), ('CYS', 'SER')]
graph edges: [('ALA', 'SER'), ('ALA', 'CYS'), ('SER', 'ALA'), ('SER', 'CYS'), ('CYS', 'ALA'), ('CYS', 'SER')]


In [36]:
_dict = graph.edges['ALA', 'CYS']

In [37]:
_dict

{'sol_htf': <perses.annihilation.relative.HybridTopologyFactory at 0x7fd26cb143c8>,
 'map_oldmol_newmol': ({},
  <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fd27462a840> >,
  <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fd27462aae0> >),
 'added_subtracted': (17.57915363479055, 20.672958966061046)}

In [39]:
zero_state_error, one_state_error = validate_endstate_energies(_dict['sol_htf']._topology_proposal, _dict['sol_htf'], _dict['added_subtracted'][0], _dict['added_subtracted'][1], beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

			HarmonicBondForce: 0.03455153971697946
			HarmonicAngleForce: 0.607201772364288
			PeriodicTorsionForce: 16.17680997925674
			NonbondedForce: -892.3403874495638
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-875.5218241582257
rp: -875.1091011234431
			CustomBondForce: 1.8647390448810859e-10
			HarmonicBondForce: 3.6920004809104694
			CustomAngleForce: 0.05177605532145648
			HarmonicAngleForce: 3.3039274116811037
			CustomTorsionForce: 4.921232087027492
			PeriodicTorsionForce: 21.200479539383966
			NonbondedForce: -888.838593781813
			CustomNonbondedForce: -2.2734923161265703
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-857.9426705234285
rp: -857.5299474886457
			CustomBondForce: 1.8647390448810859e-10
			HarmonicBondForce: 3.6920004809104694
			CustomAngleForce: 0.05177605532145648
			HarmonicAngleForce: 3.3039274116811037
			CustomTorsionForce: 4.921232087027492
			PeriodicTorsionForce: 21.200479539383966
			Nonbo

# Hooray!!!!!!!
